In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("vipoooool/new-plant-diseases-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/new-plant-diseases-dataset


In [2]:
import os
import numpy as np
import pandas as pd
from glob import glob
from PIL import Image

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
from torchvision import models


In [3]:
train_dir = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/train"
val_dir   = "/kaggle/input/new-plant-diseases-dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)/valid"
test_dir  = "/kaggle/input/new-plant-diseases-dataset/test/test"

device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device:", device)


Device: cuda


In [4]:
IMG_SIZE = 224

train_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20),
    transforms.ColorJitter(0.2, 0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

val_tf = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])


In [5]:
from torchvision.datasets import ImageFolder

train_ds = ImageFolder(train_dir, transform=train_tf)
val_ds   = ImageFolder(val_dir,   transform=val_tf)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=32, shuffle=False, num_workers=2)

idx_to_class = {v:k for k,v in train_ds.class_to_idx.items()}
num_classes = len(idx_to_class)
num_classes


38

In [6]:
model = models.mobilenet_v2(pretrained=True)

for p in model.features.parameters():
    p.requires_grad = False

model.classifier[1] = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(model.classifier[1].in_features, num_classes)
)

model = model.to(device)


/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 120MB/s]


In [7]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [8]:
for epoch in range(10):
    model.train()
    running_loss, correct = 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()

        out = model(imgs)
        loss = criterion(out, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * imgs.size(0)
        _, preds = torch.max(out, 1)
        correct += (preds == labels).sum().item()

    train_loss = running_loss / len(train_ds)
    train_acc = correct / len(train_ds)

    # validation
    model.eval()
    v_loss, v_correct = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            out = model(imgs)
            loss = criterion(out, labels)
            v_loss += loss.item() * imgs.size(0)
            _, preds = torch.max(out,1)
            v_correct += (preds == labels).sum().item()

    val_loss = v_loss / len(val_ds)
    val_acc = v_correct / len(val_ds)

    print(f"Epoch {epoch+1}: Train Acc={train_acc:.4f}  Val Acc={val_acc:.4f}")


Epoch 1: Train Acc=0.8544  Val Acc=0.9270
Epoch 2: Train Acc=0.9014  Val Acc=0.9380
Epoch 3: Train Acc=0.9071  Val Acc=0.9526
Epoch 4: Train Acc=0.9060  Val Acc=0.9472
Epoch 5: Train Acc=0.9106  Val Acc=0.9553
Epoch 6: Train Acc=0.9096  Val Acc=0.9519
Epoch 7: Train Acc=0.9107  Val Acc=0.9497
Epoch 8: Train Acc=0.9116  Val Acc=0.9516
Epoch 9: Train Acc=0.9111  Val Acc=0.9501
Epoch 10: Train Acc=0.9131  Val Acc=0.9516


In [9]:
torch.save(model.state_dict(), "mobilenetv2_plant.pth")


In [10]:
class TestDataset(Dataset):
    def __init__(self, root, transform):
        self.paths = glob(root + "/**/*", recursive=True)
        self.paths = [p for p in self.paths if p.lower().endswith((".jpg",".jpeg",".png"))]
        self.transform = transform

    def __len__(self): return len(self.paths)

    def __getitem__(self, idx):
        path = self.paths[idx]
        img = Image.open(path).convert("RGB")
        return self.transform(img), path


In [11]:
test_ds = TestDataset(test_dir, val_tf)
test_loader = DataLoader(test_ds, batch_size=32, shuffle=False)

softmax = nn.Softmax(dim=1)
records = []

def health(cls): return "Healthy" if "healthy" in cls.lower() else "Diseased"
def severity(c):
    if c < 0.5: return "Uncertain"
    if c < 0.7: return "Mild"
    if c < 0.85: return "Moderate"
    return "Severe"

def recommendation(cls):
    cls = cls.lower()
    if "healthy" in cls: return "Leaf is healthy. Good maintenance required."
    if "blight" in cls: return "Remove infected leaves, avoid overhead watering."
    if "rust" in cls: return "Improve airflow and keep foliage dry."
    if "spot" in cls: return "Improve drainage and avoid wet leaves."
    return "General disease detected. Improve air flow and remove infected parts."


In [12]:
model.eval()
with torch.no_grad():
    for imgs, paths in test_loader:
        imgs = imgs.to(device)
        out = model(imgs)
        probs = softmax(out)

        for p, confs in zip(paths, probs):
            idx = torch.argmax(confs).item()
            cls = idx_to_class[idx]
            conf = float(confs[idx])

            records.append({
                "image_path": p,
                "predicted_class": cls,
                "confidence": conf,
                "health_status": health(cls),
                "severity": severity(conf),
                "recommendation": recommendation(cls)
            })

df = pd.DataFrame(records)
df.head()
df.to_csv("dashboard_pytorch.csv", index=False)


In [100]:
df.head(10)

,image_path,predicted_class,confidence,health_status,severity,recommendation
0,/kaggle/input/new-plant-diseases-dataset/test/...,Tomato___Early_blight,0.985788,Diseased,Severe,"Remove infected leaves, avoid overhead watering."
1,/kaggle/input/new-plant-diseases-dataset/test/...,Tomato___Tomato_Yellow_Leaf_Curl_Virus,0.999766,Diseased,Severe,General disease detected. Improve air flow and...
2,/kaggle/input/new-plant-diseases-dataset/test/...,Tomato___Tomato_Yellow_Leaf_Curl_Virus,0.999999,Diseased,Severe,General disease detected. Improve air flow and...
3,/kaggle/input/new-plant-diseases-dataset/test/...,Potato___healthy,0.997922,Healthy,Severe,Leaf is healthy. Good maintenance required.
4,/kaggle/input/new-plant-diseases-dataset/test/...,Tomato___Tomato_Yellow_Leaf_Curl_Virus,0.999950,Diseased,Severe,General disease detected. Improve air flow and...
5,/kaggle/input/new-plant-diseases-dataset/test/...,Potato___Early_blight,1.000000,Diseased,Severe,"Remove infected leaves, avoid overhead watering."
6,/kaggle/input/new-plant-diseases-dataset/test/...,Tomato___Tomato_Yellow_Leaf_Curl_Virus,0.998848,Diseased,Severe,General disease detected. Improve air flow and...
7,/kaggle/input/new-plant-diseases-dataset/test/...,Tomato___Early_blight,0.819088,Diseased,Moderate,"Remove infected leaves, avoid overhead watering."
8,/kaggle/input/new-plant-diseases-dataset/test/...,Corn_(maize)___Common_rust_,1.000000,Diseased,Severe,Improve airflow and keep foliage dry.
9,/kaggle/input/new-plant-diseases-dataset/test/...,Apple___Apple_scab,0.998815,Diseased,Severe,General disease detected. Improve air flow and...


In [127]:
import torch
from torchvision import models
import torch.nn as nn

device = "cuda" if torch.cuda.is_available() else "cpu"

model = models.mobilenet_v2(weights=None)
model.classifier[1] = nn.Sequential(
    nn.Dropout(0.2),
    nn.Linear(model.classifier[1].in_features, 38)
)

model.load_state_dict(torch.load("/kaggle/working/mobilenetv2_plant.pth", map_location=device))
model.to(device)
model.eval()
print("Model Loaded!")


Model Loaded!


In [128]:
from torchvision import transforms
from PIL import Image

tf = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])


In [129]:
idx_to_class = {v:k for k,v in train_ds.class_to_idx.items()}


In [130]:
def predict_image(img_path):
    img = Image.open(img_path).convert("RGB")
    x = tf(img).unsqueeze(0).to(device)

    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1)[0]
        idx = torch.argmax(probs).item()

    cls = idx_to_class[idx]
    confidence = float(probs[idx])

    print("Prediction:", cls)
    print("Confidence:", round(confidence, 4))

    return cls, confidence


In [ ]:
predict_image("/kaggle/working/strawberry_scrotch.jpeg")
predict_image("/kaggle/working/tomato_blight_new.jpeg")
predict_image("/kaggle/working/corn_image.jpeg")
predict_image("/kaggle/working/apple-6.png")

predict_image("/kaggle/working/tomato.jpeg")


Prediction: Strawberry___Leaf_scorch
Confidence: 0.9954
Prediction: Tomato___Late_blight
Confidence: 1.0
Prediction: Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot
Confidence: 0.9991
Prediction: Apple___Apple_scab
Confidence: 1.0
Prediction: Tomato___Septoria_leaf_spot
Confidence: 0.7462


('Tomato___Septoria_leaf_spot', 0.7461826801300049)

In [147]:
val_loader
idx_to_class


{0: 'Apple___Apple_scab',
 1: 'Apple___Black_rot',
 2: 'Apple___Cedar_apple_rust',
 3: 'Apple___healthy',
 4: 'Blueberry___healthy',
 5: 'Cherry_(including_sour)___Powdery_mildew',
 6: 'Cherry_(including_sour)___healthy',
 7: 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot',
 8: 'Corn_(maize)___Common_rust_',
 9: 'Corn_(maize)___Northern_Leaf_Blight',
 10: 'Corn_(maize)___healthy',
 11: 'Grape___Black_rot',
 12: 'Grape___Esca_(Black_Measles)',
 13: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)',
 14: 'Grape___healthy',
 15: 'Orange___Haunglongbing_(Citrus_greening)',
 16: 'Peach___Bacterial_spot',
 17: 'Peach___healthy',
 18: 'Pepper,_bell___Bacterial_spot',
 19: 'Pepper,_bell___healthy',
 20: 'Potato___Early_blight',
 21: 'Potato___Late_blight',
 22: 'Potato___healthy',
 23: 'Raspberry___healthy',
 24: 'Soybean___healthy',
 25: 'Squash___Powdery_mildew',
 26: 'Strawberry___Leaf_scorch',
 27: 'Strawberry___healthy',
 28: 'Tomato___Bacterial_spot',
 29: 'Tomato___Early_blight',
 30: '

In [148]:
all_preds = []
all_labels = []

model.eval()
with torch.no_grad():
    for images, labels in val_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)

        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())


In [150]:
from sklearn.metrics import classification_report

class_names = [idx_to_class[i] for i in range(len(idx_to_class))]

print(classification_report(all_labels, all_preds, target_names=class_names))


                                                    precision    recall  f1-score   support

                                Apple___Apple_scab       0.90      0.99      0.94       504
                                 Apple___Black_rot       0.99      0.98      0.99       497
                          Apple___Cedar_apple_rust       1.00      0.92      0.96       440
                                   Apple___healthy       0.99      0.97      0.98       502
                               Blueberry___healthy       0.94      0.98      0.96       454
          Cherry_(including_sour)___Powdery_mildew       0.98      0.98      0.98       421
                 Cherry_(including_sour)___healthy       0.99      1.00      0.99       456
Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot       0.97      0.76      0.85       410
                       Corn_(maize)___Common_rust_       0.99      0.99      0.99       477
               Corn_(maize)___Northern_Leaf_Blight       0.83      0.97      0.